In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('DataFrame_Claims_Spain.csv', encoding='latin_1', index_col=0,
                 parse_dates=['Claim Incident date', 'FE_Declaration_date', 'Initial coverage date',
                              'First claim decision date', 'Last claim decisión date', 'Policy Holder date of birth'],
                              infer_datetime_format=True, dtype={'Age at signature': np.int64})

In [ ]:
df.head()

The column `'Policy Holder date of birth'` contain a corrupted date

In [ ]:
df['Policy Holder date of birth'][df['Policy Holder date of birth'].str.contains('1070')]

In [ ]:
df.loc[33949, 'Policy Holder date of birth'] = '19/05/1970'
df.loc[33949, 'Age at signature'] = 29

Convert the column containing some date to the `datetime` dtype

In [ ]:
df['Policy Holder date of birth'] = pd.to_datetime(df['Policy Holder date of birth'])

Convert to category the categorical columns

In [ ]:
def convert_to_int_object(col):
    serie = []
    for _, x in col.iteritems():
        try:
            serie.append(int(x))
        except ValueError:
            serie.append(np.nan)
    return pd.Series(serie, index=col.index, dtype=object)

In [ ]:
df['Risk code'] = df['Risk code'].astype('category')
df['Sexo'] = df['Sexo'].astype('category')
df['Refused decision reason code'] = convert_to_int_object(df['Refused decision reason code']).astype('category')
df['Trad_Refusal_reason'] = df['Trad_Refusal_reason'].astype('category')
df['Refusal_Category'] = df['Refusal_Category'].astype('category')
df['Claim_Status_Level_0']= df['Claim_Status_Level_0'].astype('category')
df['Refusal_Flag'] = df['Refusal_Flag'].astype('category')
df['Local Partner name categories'] = df['Local Partner name categories'].astype('category')
df['Insured NIF categories'] = df['Insured NIF categories'].astype('category')

Drop the column to not consider during classification

In [ ]:
target = df['Refusal_Flag']
data = df.drop(columns=[
    'Refusal_Flag',
    'Refused decision reason code',
    'Claim_Status_Level_0',
    'Trad_Refusal_reason',
    'Refusal_Category'
])

Encode the label

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label = label_encoder.fit_transform(target)

Define the different part of the pipeline:

* One-hot encode the categorical features;
* Ordinal encode the binary categorical features;
* Standard scale the numerical features;
* Feature engineer the date by creating: (i) the time to declaration and (ii) the contract seniority.

In [ ]:
one_hot_encoded_cat_features = ['Risk code',
                                'Local Partner name categories',
                                'Insured NIF categories',
                                'Claim Number categories']
ordinal_encoded_features = ['Sexo']
standard_scaled_features = ['Insured amount', 'Initial_Instalment_Amount',
                            'Age at signature']
time_to_declaration_features = ['FE_Declaration_date', 'Claim Incident date']
contract_seniority_features = ['Claim Incident date', 'Initial coverage date']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer

from sklearn.impute import SimpleImputer

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from function_FunctionTransformer import time_to_declaration
from function_FunctionTransformer import contract_seniority

In [ ]:
preprocessor = make_column_transformer(
    (ordinal_encoded_features, OrdinalEncoder()),
    (one_hot_encoded_cat_features, make_pipeline(
        SimpleImputer(strategy='constant', fill_value=-1),
        OneHotEncoder(handle_unknown='ignore'))),
    (time_to_declaration_features, make_pipeline(
        FunctionTransformer(func=time_to_declaration, validate=False),
        MinMaxScaler())),
    (contract_seniority_features, make_pipeline(
        FunctionTransformer(func=contract_seniority, validate=False),
        MinMaxScaler())),
    (standard_scaled_features, make_pipeline(
        StandardScaler(),
        SimpleImputer(strategy='median'))),
    n_jobs=-1
)

Use a RandomForestClassifier to make some classification within a 3-fold cross-validation. We will return the `balanced_accuracy_score` and the `roc_auc_score`.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

In [ ]:
pipe = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=100, n_jobs=-1))

In [ ]:
scores = cross_validate(pipe, data, label, scoring=['roc_auc', 'balanced_accuracy'], cv=3, n_jobs=-1)

Convert the scores to a dataframe to have a nice display

In [ ]:
scores = pd.DataFrame(scores)
scores

Compute the mean performance

In [ ]:
scores.mean().to_frame().T

As well as the std. dev. of those performance

In [ ]:
scores.std().to_frame().T